In [1]:
import matplotlib.pyplot as plt
import numpy as np
from CA_model import CA_model
import evaluation
import iceing
import scipy.stats as stats
from skimage.transform import resize

In [2]:
iceing_test = iceing.iceing_model(0.45, 100)

In [3]:
ponds, sums = iceing_test.run(15_000_000)

In [4]:
areas_ising, perimeters_ising = evaluation.perim_area(ponds, pond_val = -1, ice_val = 1)

In [5]:
# replacement for CA model
size = 100
h_norm = np.random.normal(loc = 0.5, scale = 1, size = (int(size/3), int(size/3)))
h_norm = resize(h_norm, (size, size))
h_norm = np.heaviside(h_norm-0.6, 0) * h_norm

areas_norm, perimeters_norm = evaluation.perim_area(np.where(h_norm>0, -1, 1), pond_val = -1, ice_val = 1)
shapes_norm= np.log(areas_norm)/np.log(perimeters_norm)

In [6]:
shapes_ising = np.log(areas_ising)/np.log(perimeters_ising)

In [7]:
stats.ks_2samp(np.log(areas_norm),np.log(areas_ising))

KstestResult(statistic=0.15384615384615385, pvalue=0.6206865585848859, statistic_location=1.0986122886681098, statistic_sign=1)

In [8]:
def my_statistic(x, y):
    return np.mean(x) - np.mean(y)

stats.permutation_test((shapes_ising,shapes_norm),statistic=my_statistic)

PermutationTestResult(statistic=0.029381242312498057, pvalue=0.5232, null_distribution=array([ 0.05924845, -0.00820734, -0.05705156, ..., -0.06789756,
       -0.06651877,  0.01994486]))

In [ ]:
sample1 = areas_ising/perimeters_ising
sample2 = areas_norm/perimeters_norm

plt.figure(figsize=(8, 6))
plt.hist(sample1, bins=20, density=True, alpha=0.5, label='Sample 1')
plt.hist(sample2, bins=20, density=True, alpha=0.5, label='Sample 2')

# Plot the PDFs
x = np.linspace(-4, 4, 100)
pdf_sample1 = stats.norm.pdf(x, np.mean(sample1), np.std(sample1))
pdf_sample2 = stats.norm.pdf(x, np.mean(sample2), np.std(sample2))
plt.plot(x, pdf_sample1, 'r-', label='PDF Sample 1')
plt.plot(x, pdf_sample2, 'b-', label='PDF Sample 2')

# Add labels and a legend
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.title('PDF Comparison')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.scatter(areas_norm, perimeters_norm, marker='x', label = 'normal distribution')
plt.scatter(areas_ising, perimeters_ising, marker='.', alpha=0.2, label = 'Ising')
plt.legend()
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel('area [m^2]')
plt.ylabel('perimeter [m]')